<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">INFO</h1>

In this script, we create a class that will structure the unit tests for the `DFS` player. \
We choose to use the `unittest` library. \
Then, we run them to ensure that all methods developed work as expected.

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">IMPORTS</h1>

In [1]:
# External imports
from typing import *
from typing_extensions import *
from numbers import *
import unittest
import sys
import os
import random
import itertools

# Add needed directories to the path
sys.path.append(os.path.join("..", "players"))

# PyRat imports
from SortedNeighbors import SortedNeighbors
from pyrat import BigHolesRandomMaze, Action
from pyrat import Player, Maze, GameState, Action,Graph

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">DEFINE THE TESTS</h1>

The `unittest` library requires the creation of a class that extends `unittest.TestCase`. \
For each method to test, we need to define a method in the test class. \
Each of these test methods should call the tested method with various inputs to check that produced outputs match expected ones.

In [2]:
class SortedNeighborsTests (unittest.TestCase):

    """
        This class tests the methods of the SortedNeighbors class.
        For each method, we test it with a few different configurations.
    """

    #############################################################################################################################################
    #                                                                 UNIT TESTS                                                                #
    #############################################################################################################################################

    def test_maze_to_graph(self: Self) -> None:
        """
        This method tests the 'maze_to_graph' method of the SortedNeighbors class.
        We are going to check the following:
        - Outputs are of the expected types.
        - All vertices are included in the graph.
        - All edges are included in the graph.
        - The graph is correctly constructed from the maze.
        In:
            * self: Reference to the current object.
        Out:
            * None.
        """

        # Constants
        NB_GRAPHS = 10
        WIDTHS = [2, 30]
        HEIGHTS = [2, 30]
        CELL_PERCENTAGES = [20.0, 100.0]
        WALL_PERCENTAGES = [20.0, 100.0]
        MUD_PERCENTAGE = 0.0

        # Test on several graphs
        for i in range(NB_GRAPHS):
            
            # Instantiate the player
            player = SortedNeighbors()

            # Generate a random maze
            # We use a fixed random seed for reproducibility of tests
            rng = random.Random(i)
            maze = BigHolesRandomMaze(width = rng.randint(WIDTHS[0], WIDTHS[1]),
                                      height = rng.randint(HEIGHTS[0], HEIGHTS[1]),
                                      cell_percentage = rng.uniform(CELL_PERCENTAGES[0], CELL_PERCENTAGES[1]),
                                      wall_percentage = rng.uniform(WALL_PERCENTAGES[0], WALL_PERCENTAGES[1]),
                                      mud_percentage = MUD_PERCENTAGE,
                                      random_seed = i)
            

            # Convert maze to graph
            # Create a dummy game state with necessary attributes
            class DummyGameState:
                def __init__(self):
                    self.player_locations = {player.name: (0, 0)}
                    self.cheese = []

            graph, path_dict = player.maze_to_graph(maze, game_state=DummyGameState())

            # Check the output type for graph
            # It should be a tuple with two elements : a graph and a dictionary
            self.assertTrue(isinstance(graph, Graph))
            self.assertTrue(isinstance(path_dict, Dict))

            #the dictionnaire should have the following structure
            self.assertTrue(all(isinstance(v, Dict) for v in path_dict.values()))
            for d in path_dict.values():
                self.assertTrue(all(isinstance(k, Integral) for k in d.keys()))  
                self.assertTrue(all(isinstance(v, list) for v in d.values()))    
                self.assertTrue(all(isinstance(k, Integral) for k in d.keys()))  
            
            
            # All vertices of the maze should be included in the graph
            for vertex in maze.vertices:
                self.assertIn((vertex, vertex), graph.vertices)
            

            # All edges should be included in the graph
            for vertex in maze.vertices:
                if vertex in graph.edges:
                    for neighbor in maze.get_neighbors(vertex):
                        self.assertIn(neighbor, graph.edges[vertex])

            # Check that maze_to_graph[1] returns a dictionary of dictionaries which for each pair of vertices returns the best path connecting these vertices
            for start_vertex in path_dict:
                for end_vertex in path_dict[start_vertex]:
                    path = path_dict[start_vertex][end_vertex]
                    self.assertTrue(isinstance(path, List))
                    self.assertTrue(all(isinstance(vertex, Integral) for vertex in path))
                    self.assertEqual(path[0], start_vertex)
                    self.assertEqual(path[-1], end_vertex)
                    for j in range(len(path) - 1):
                        self.assertIn(path[j + 1], graph.edges[path[j]])
            



    def test_route_mouse(self:Self) -> None:
        """
        This method tests the 'route_mouse' method of the SortedNeighbors class.
        We are going to check the following:
        - Outputs are of the expected types.
        - The path is correctly computed.
        In:
            * self: Reference to the current object.
        Out:
            * None.
        """

        # Constants
        NB_GRAPHS = 10
        WIDTHS = [2, 30]
        HEIGHTS = [2, 30]
        CELL_PERCENTAGES = [20.0, 100.0]
        WALL_PERCENTAGES = [20.0, 100.0]
        MUD_PERCENTAGE = 0.0

        # Test on several graphs
        for i in range(NB_GRAPHS):
            
            # Instantiate the player
            player = SortedNeighbors()

            # Generate a random maze
            # We use a fixed random seed for reproducibility of tests
            rng = random.Random(i)
            maze = BigHolesRandomMaze(width = rng.randint(WIDTHS[0], WIDTHS[1]),
                                      height = rng.randint(HEIGHTS[0], HEIGHTS[1]),
                                      cell_percentage = rng.uniform(CELL_PERCENTAGES[0], CELL_PERCENTAGES[1]),
                                      wall_percentage = rng.uniform(WALL_PERCENTAGES[0], WALL_PERCENTAGES[1]),
                                      mud_percentage = MUD_PERCENTAGE,
                                      random_seed = i)
            

            
            # Convert maze to graph
            # Create a dummy game state with necessary attributes
            class DummyGameState:
                def __init__(self):
                    self.player_locations = {player.name: (0, 0)}
                    self.cheese = []

            dummy_game_state = DummyGameState()


            path_dict = player.maze_to_graph(maze, game_state=dummy_game_state)[1]
            graph = player.maze_to_graph(maze, game_state=dummy_game_state)[0]


            # Create a random route to test
            start = random.choice(list(path_dict.keys()))
            if path_dict[start]:
                end = random.choice(list(path_dict[start].keys()))
                route = path_dict[start][end]
            else:
                continue

            # Compute the path
            path = player.route_mouse(graph, route)

            # Check the output type for path
            # It should be a list of integers
            self.assertTrue(isinstance(path, List))
            self.assertTrue(all(isinstance(vertex, Integral) for vertex in path))

            # Check that the path is correct
            current_vertex = start
            for vertex in path:
                self.assertIn(vertex, graph[current_vertex])
                current_vertex
            
          

             
    def test_sorted_neighbors(self: Self) -> None:
        """
        This method tests the 'sorted_neighbors' method of the SortedNeighbors class.
        We are going to check the following:
        - Outputs are of the expected types.
        - The path is correctly computed.
        - The algorithm stops as soon as the path being created is longer than the shortest path found so far.
        -the code sorts the vertices before visiting them
        In:
            * self: Reference to the current object.
        Out:
            * None.
        """

        # Constants
        NB_GRAPHS = 10
        WIDTHS = [2, 30]
        HEIGHTS = [2, 30]
        CELL_PERCENTAGES = [20.0, 100.0]
        WALL_PERCENTAGES = [20.0, 100.0]
        MUD_PERCENTAGE = 0.0

        # Test on several graphs
        for i in range(NB_GRAPHS):
                    
            # Instantiate the player
            player = SortedNeighbors()

            # Generate a random maze
            # We use a fixed random seed for reproducibility of tests
            rng = random.Random(i)
            maze = BigHolesRandomMaze(width = rng.randint(WIDTHS[0], WIDTHS[1]),
                                      height = rng.randint(HEIGHTS[0], HEIGHTS[1]),
                                      cell_percentage = rng.uniform(CELL_PERCENTAGES[0], CELL_PERCENTAGES[1]),
                                      wall_percentage = rng.uniform(WALL_PERCENTAGES[0], WALL_PERCENTAGES[1]),
                                      mud_percentage = MUD_PERCENTAGE,
                                      random_seed = i)
                    
            # Convert maze to graph
            # Create a dummy game state with necessary attributes
            class DummyGameState:
                def __init__(self):
                    self.player_locations = {player.name: (0, 0)}
                    self.cheese = []

            dummy_game_state = DummyGameState()

            graph, path_dict = player.maze_to_graph(maze, game_state=dummy_game_state)

            # Choose a random vertex to start
            start = random.choice(list(path_dict.keys()))

            # Compute the TSP path
            result = player.SortedNeighbors(graph, start)
            if isinstance(result, tuple) and len(result) == 2:
                tsp_path, min_distance = result
            else:
                tsp_path = result
                min_distance = sum(graph[tsp_path[j]][tsp_path[j + 1]] for j in range(len(tsp_path) - 1))  # calculate min_distance if not provided

            # Check the output type for tsp_path
            # It should be a list of integers
            self.assertTrue(isinstance(tsp_path, List))
            #self.assertTrue(all(isinstance(vertex, Integral) for vertex in tsp_path))

            # Test that the code sorts the vertices before visiting them
            sorted_vertices = player.SortedNeighbors(graph, start)
            self.assertEqual(sorted_vertices, sorted(sorted_vertices))

            # Check that the TSP path visits every vertex exactly once
            self.assertEqual(len(set(tsp_path)), len(graph.vertices))

            # Check that the TSP path is of minimal distance
            total_distance = sum(graph[tsp_path[j]][tsp_path[j + 1]] for j in range(len(tsp_path) - 1))
            self.assertEqual(total_distance, min_distance)

            # Ensure the path starts and ends at the start vertex
            self.assertEqual(tsp_path[0], start)
            self.assertEqual(tsp_path[-1], start)

            # Check that the algorithm stops as soon as the path being created is longer than the shortest path found so far
            for perm in itertools.permutations(graph.vertices):
                if perm[0] == start:
                    distance = 0
                    for j in range(len(perm) - 1):
                        distance += graph[perm[j]][perm[j + 1]]
                        if distance > min_distance:
                            break
                    else:
                        self.assertEqual(distance, min_distance)





<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">RUN THE TESTS</h1>
           
When calling `unittest.main()`, all methods in the test class above will be run.

In [3]:
# Run all tests
_ = unittest.main(argv=[""], verbosity=2, exit=False)

test_maze_to_graph (__main__.SortedNeighborsTests.test_maze_to_graph)
This method tests the 'maze_to_graph' method of the SortedNeighbors class. ... FAIL
test_route_mouse (__main__.SortedNeighborsTests.test_route_mouse)
This method tests the 'route_mouse' method of the SortedNeighbors class. ... 

Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor


ok
test_sorted_neighbors (__main__.SortedNeighborsTests.test_sorted_neighbors)
This method tests the 'sorted_neighbors' method of the SortedNeighbors class. ... 

Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor


ok


Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor



FAIL: test_maze_to_graph (__main__.SortedNeighborsTests.test_maze_to_graph)
This method tests the 'maze_to_graph' method of the SortedNeighbors class.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\rouzi\AppData\Local\Temp\ipykernel_11208\774451282.py", line 75, in test_maze_to_graph
    self.assertIn((vertex, vertex), graph.vertices)
AssertionError: (1, 1) not found in [(0, 0)]

----------------------------------------------------------------------
Ran 3 tests in 3.864s

FAILED (failures=1)
